<a href="https://colab.research.google.com/github/ZhexuanWang/AI-model-training/blob/master/StableDiffusion%2BLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🗂️ 挂载 Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 🧰 安装依赖
!pip install --upgrade diffusers transformers accelerate safetensors datasets
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install -e .

# 🔐 登录 Hugging Face（需要访问 token）
from huggingface_hub import notebook_login
notebook_login()

# 📁 设置路径和参数
pretrained_model = "stabilityai/stable-diffusion-2-1"
instance_data_dir = "/content/TrainingPics"  # 你的像素图数据集路径
output_dir = "/content/drive/MyDrive/pixel_lora_model"
instance_prompt = "Pixel Game Character Chibi"  # 你想训练的风格关键词

# 🚀 启动训练（LoRA 微调）
!accelerate launch examples/dreambooth/train_dreambooth_lora.py \
  --pretrained_model_name_or_path=$pretrained_model \
  --instance_data_dir=$instance_data_dir \
  --output_dir=$output_dir \
  --instance_prompt="$instance_prompt" \
  --resolution=320 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --max_train_steps=1000 \
  --use_8bit_adam \
  --mixed_precision="fp16" \
  --enable_xformers_memory_efficient_attention \
  --checkpointing_steps=500 \
  --seed=42

from diffusers import StableDiffusionPipeline
import torch

base_model = "stabilityai/stable-diffusion-2-1"
lora_path = "/content/drive/MyDrive/pixel_lora_model"

pipe = StableDiffusionPipeline.from_pretrained(
    base_model,
    torch_dtype=torch.float16
).to("cuda")

pipe.load_lora_weights(lora_path)

prompt = "pixel game character chibi knight with sword"
image = pipe(prompt).images[0]
image.save("/content/drive/MyDrive/pixel_knight.png")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'diffusers' already exists and is not an empty directory.
/content/diffusers
Obtaining file:///content/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.36.0.dev0-0.editable-py3-none-any.whl size=11367 sha256=8a98e298020f9a396003ee729073023d9df9b45f4637f5004cd4ca6db3967864
  Stored in directory: /tmp/pip-ephem-wheel-cache-j886zplp/wheels/8a/fc/09/385efb77b455b2fd4a656c950079c93147e1f50ae614e51beb
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.36.0.dev0
    Uninstalling diffusers-0.36.0.dev0:
      Suc

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-09-29 15:50:10.370378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759161010.390655    8672 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759161010.396845    8672 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759161010.412635    8672 computation_plac

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
!pip install bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.4 MB/s eta 0:00:00


In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
from PIL import ImageFilter, ImageEnhance

def pixelate(img, pixel_size=16, output_size=(100, 100)):
    # 缩小图像到低分辨率
    small = img.resize(
        (output_size[0] // pixel_size, output_size[1] // pixel_size),
        resample=Image.NEAREST
    )
    # 再放大回目标尺寸，使用 NEAREST 保留块状感
    result = small.resize(output_size, Image.NEAREST)
    return result

def quantize_fixed_palette(img):
    palette_img = Image.new("P", (1, 1))
    palette_img.putpalette([
        # 16色调色板（RGB，每组3个值）
        0, 0, 0,       # 黑
        255, 255, 255, # 白
        255, 0, 0,     # 红
        0, 255, 0,     # 绿
        0, 0, 255,     # 蓝
        255, 255, 0,   # 黄
        255, 0, 255,   # 紫
        0, 255, 255,   # 青
        128, 128, 128, # 灰
        64, 64, 64,    # 深灰
        192, 192, 192, # 浅灰
        128, 0, 0,     # 深红
        0, 128, 0,     # 深绿
        0, 0, 128,     # 深蓝
        128, 128, 0,   # 橄榄
        128, 0, 128    # 深紫
    ] + [0] * (768 - 48))  # 补满768项
    return img.convert("RGB").quantize(palette=palette_img).convert("RGB")

# 加载原始模型
pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch.float16
).to("cuda")

# 加载 LoRA adapter（自动识别 .safetensors）
pipe.load_lora_weights("/content/drive/MyDrive/pixel_lora_model")

# 推理
prompt = "pixel character chibi rogue, 8-bit style, vibrant colors, saturated palette, no shading"
image = pipe(prompt).images[0]
image = ImageEnhance.Color(image).enhance(1.8)  # 增强饱和度
image = image.filter(ImageFilter.EDGE_ENHANCE)
image = quantize_fixed_palette(image)
pixel_image = pixelate(image, pixel_size=16, output_size=(100, 100))
image.save("/content/drive/MyDrive/pixel_rogue__100x100.png")


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


  0%|          | 0/50 [00:00<?, ?it/s]